# Interactive Plotting with Jupyter

There are several ways to interactively plot. In this tutorial I will show how to interact with 2D and 1D data.  There are other ways to interact with large tables of data using either [Bokeh](https://docs.bokeh.org/en/latest/index.html) (shown the Skyfit notebook) or [Glue](http://docs.glueviz.org/en/stable). A non-python based solution that also works with large tables of data is Topcat. 

Most of the methods here will work on the command line.  In order to make this work within Jupyter you will need to use the "magic" command `%matplotlib notebook`. Additionally, you may need to install the `ipywidgets` module using:

    conda install ipywidgets

More info about the ipywidgets (which we use below) is available here:

https://ipywidgets.readthedocs.io/


In [ ]:
import sys
import astropy
import astroquery
import ipywidgets
import matplotlib

print('\n Python version: ', sys.version)
print('\n Astropy version: ', astropy.__version__)
print('\n Matplotlib version: ', matplotlib.__version__)
print('\n Astroquery version: ', astroquery.__version__)
print('\n ipywidgets version: ', ipywidgets.__version__)

In [ ]:
import glob,os,sys

import numpy as np
import matplotlib.pyplot as plt 

import astropy.io.fits as pyfits
import astropy.units as u
from astroquery.skyview import SkyView

import ipywidgets as widgets

Here we need an image to play with, we can either download it via SkyView or load one from our machine.

In [ ]:
ext = 0

# download an image
pflist = SkyView.get_images(position='M82', survey=['SDSSr'], radius=10 * u.arcmin)
pf = pflist[0] # first element of the list, might need a loop if multiple images

# or load an image
#pf = pyfits.open('m82.fits')

image = pf[ext].data

Next we need to turn on the interactive plotting.  

In [ ]:
# turn-on interactive plots
%matplotlib notebook

# Display an image (2D data)

We plot a 2D image using imshow, we can set the scale of the image as well as the colormap.

In [ ]:
fig = plt.figure()
plt.ion()

p = fig.add_subplot(111)
p.imshow(image, interpolation='Nearest', origin='lower', vmin=-10, vmax=20, cmap='viridis')

plt.show()

# Add an event to the display

There are several types of matplotlib events that you can use to interact with a figure. 

A few useful events are the following:

`button_press_event` 	
`button_release_event` 	
`key_press_event`  
`key_release_event`  

For more information on event handling and examples check out the following website: 
https://matplotlib.org/stable/users/event_handling.html

Here we add a python function (usually referred to as a *callback* function) linking to the `key_press_event`. The function checks for which key is being pressed and, if the condition is met, runs its code: plotting a red point on the image whenever the 'm' key is pressed.  We can easily add more keys adding more functionaly to our interactive figure.

Note that our function is never explicitly called in our script, but rather called by something called the *event loop*. This is an example of asynchronous programming, and can take some getting used to.

In [ ]:
fig = plt.figure()
plt.ion()

p = fig.add_subplot(111)
p.imshow(image, interpolation='Nearest', origin='lower', vmin=-10, vmax=20, cmap='viridis')

def on_key_press(event):
    xc, yc = event.xdata, event.ydata

    if event.key == 'm':
        p.plot(xc,yc,'ro', markersize=5)
        fig.canvas.draw_idle

# This associates the event to the callback function
fig.canvas.mpl_connect('key_press_event', on_key_press)

plt.show()

# Add output to the display with the event

If we want to display the coordinate of the points we mark, we need to use the Output widget. We need this special widget to get asynchronous output in a notebook. If you are using the command-line, you could simply use `print()` to get information to the terminal.

There's a new bit of python syntax in this cell:  `@out.capture()`. The `@` symbol indicates a function *decorator*. Essentially, decorators are a way to give python functions special properties. In this case, any output that `on_key_press` creates (`print` output, raised exceptions, etc) will be captured and output to the Output widget.

In [ ]:
#plt.ioff()
fig = plt.figure(figsize=[6,6])
plt.ion()

p = fig.add_subplot(111)
p.imshow(image, interpolation='Nearest', origin='lower', vmin=-10, vmax=20, cmap='viridis')

out = widgets.Output()
@out.capture()
def on_key_press(event):
    xc, yc = event.xdata, event.ydata

    if event.key == 'm':
        p.plot(xc,yc,'ro', markersize=5)
        fig.canvas.draw_idle
        
        print("[%.1f, %.1f] = %.4f" % (xc, yc, image[int(yc),int(xc)]))

fig.canvas.mpl_connect('key_press_event', on_key_press)

display(out)


# Interactive 1D data

Let's use some more of the widgets. Key press and mouse events are good for indicating locations in the plotting area, but what about changing the value of some variable? For this, widgets are very useful. In the next example, we setup a plot that shows a slice of the 2D data above (a column of pixels). But it would be nice to select *which* column. For this we use a slider. First, let's setup the graph, then define the *callback* function `update`. Instead of an event, though, the function will get an object `change` that has the new (`change.new`) and old (`change.old`) values.

Lastly, we make the slider object with some properties and use its `observe` member function of register the `update` callback to changes in its value.

In [ ]:
zl,xl = image.shape

fig = plt.figure(figsize=[6,6])
p = fig.add_subplot(111)
#p.set_yscale('log')

slice = image[150,:]
line, = p.plot(slice)

def update(change):
    # Change the line's data to the new column
    line.set_ydata(image[change.new,:])
    # because the data has changed, force a re-draw
    fig.canvas.draw()

int_slider = widgets.IntSlider(
    value=150,
    min=0,
    max=zl,
    step=1,
    description='Z-axis:',
    continuous_update=True
)
int_slider.observe(update, 'value')
int_slider

# Another Example

This time, we'll query the Sload Digital Sky Survey (SDSS) catalog server for spectra of objects around a certain point on the sky. The following code with use `astroquery` to do this and we'll end up with a list of spectra `sp`. Each spectrum will be a `FITS` object (see [astropy.io.fits](https://docs.astropy.org/en/stable/io/fits/index.html) documentation. These `FITS` objects are also like lists, and the 2nd element (or extension in FITS-speak) has a table with the log-wavelengths (`loglam`) and flux as columns.

In [ ]:
from astroquery.sdss import SDSS
from astropy import coordinates

# Coordinates of the sky
ra, dec = 148.969687, 69.679383
# Convert to astropy.coordinates.SkyCoord for use with astroquery
co = coordinates.SkyCoord(ra=ra, dec=dec,unit=(u.deg, u.deg), frame='fk5')
xid = SDSS.query_region(co, radius=20 * u.arcmin, spectro=True)
sp = SDSS.get_spectra(matches=xid)
print("N =",len(sp))

If you didn't change the coordinates, you there should be 10 spectra. We could plot each one out, but for this example, let's plot a single one at a time and use a slider to select which one of the 10.

In [ ]:
ext = 1           # The extension that has the spectrum
n_max = len(sp)-1 # total number of spectra - 1

pf = sp[0]
tab = pf[ext].data

spec = tab['flux']
wave = 10**tab['loglam']

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
line, = ax.plot(wave,spec)
ax.set_xlabel('Wavelength [Angstroms]')
ax.set_ylabel('Flux')

def new_spec(change):
    pf = sp[change.new]
   
    pf[ext].header
    tab = pf[ext].data

    spec = tab['flux']
    wave = 10**tab['loglam']
    
    line.set_xdata(wave)
    line.set_ydata(spec)
    fig.canvas.draw()

int_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=n_max,
    step=1,
    description='Spectrum:',
    continuous_update=True
)
int_slider.observe(new_spec, 'value')
int_slider

# Another widget

Let's do the same thing again, but this time, we add another widget:  one that simply holds a True/False value. If True, we'll plot a vertical dashed line representing the location of Hydrogen-Alpha (H$\alpha$), at $\lambda = 6563$ Angstroms.

In [ ]:
ext = 1
n_max = len(sp)-1 # total number of spectra - 1

pf = sp[0]
pf[ext].header
tab = pf[ext].data

spec = tab['flux']
wave = 10**tab['loglam']

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
line, = ax.plot(wave,spec)
ax.set_xlabel('Wavelength [Angstroms]')
ax.set_ylabel('Flux')

# This vertical line is always "there", but set it invisible.
line2, = ax.plot([6563,6563],[0,20],"--",c="r")
line2.set_visible(False)

def new_spec(change):
    pf = sp[change.new]
   
    pf[ext].header
    tab = pf[ext].data

    spec = tab['flux']
    wave = 10**tab['loglam']
    
    line.set_xdata(wave)
    line.set_ydata(spec)
    fig.canvas.draw()
    
    
def display_lines(change):
    if change.new: line2.set_visible(True)
    else: line2.set_visible(False)
    fig.canvas.draw()

int_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=n_max,
    step=1,
    description='Spectrum:',
    continuous_update=False
)
int_slider.observe(new_spec, 'value')
display(int_slider)


    
chk_box = widgets.Checkbox(
    value=False,
    description='Line list',
)

chk_box.observe(display_lines, 'value')
display(chk_box)

In [ ]:
# turn-off interactive plots
%matplotlib inline 

# Resources

https://ipywidgets.readthedocs.io/

https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html

https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html

https://kapernikov.com/ipywidgets-with-matplotlib/

https://matplotlib.org/stable/users/event_handling.html

https://docs.bokeh.org/en/latest/index.html

http://docs.glueviz.org/en/stable